# Graph learning from data

* Using a scipy.optimize.minimize routine to learn the graph structure using synthetic data, this is based on one of the methods discussed in arxiv:1601.02513
* $ min \ \ \ tr(X^{T}LX) +\alpha ||L||_{F}$

In [1]:
from minimizer import *

* Specify number of sensors and alpha (1t = 127 sensors)

In [2]:
n_sensors = 127

* Load synthetic data for 1T detector config

In [3]:
with open("optical_1t.pickle", "rb") as f:
    signal = pickle.load(f)

* Using trace's cyclic property we can calculate the product $X^{T}X$, where $S=X^{T}X$ 

In [4]:
S = (1.0/len(signal))*np.matmul(np.asarray(signal).T,np.asarray(signal))

* Use the identity matrix times 2 a $L_{0}$ (first guess)

In [5]:
L0 = np.identity(n_sensors)*2

In [6]:
L0

array([[2., 0., 0., ..., 0., 0., 0.],
       [0., 2., 0., ..., 0., 0., 0.],
       [0., 0., 2., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 2., 0., 0.],
       [0., 0., 0., ..., 0., 2., 0.],
       [0., 0., 0., ..., 0., 0., 2.]])

* Call minimization routine
* Input parameters:
    * number of sensors
    * alpha (a.k.a inverse of graph density)
    * S = $X^{T}X$

In [7]:
myMinimizer = Minimizer(n_sensors=n_sensors, alpha=0.33, S=S)

In [8]:
sol = myMinimizer.Optimization(L0=L0, maxiter=100)

| niter |f evals|CG iter|  obj func   |tr radius |   opt    |  c viol  | penalty  |barrier param|CG stop|
|-------|-------|-------|-------------|----------|----------|----------|----------|-------------|-------|
|   1   | 16130 |   0   | +1.8250e+05 | 1.00e+00 | 8.08e+02 | 0.00e+00 | 1.00e+00 |  1.00e-01   |   0   |
|   2   | 32260 |   1   | +1.6838e+05 | 7.00e+00 | 8.03e+02 | 0.00e+00 | 1.00e+00 |  1.00e-01   |   2   |
|   3   | 48390 |   2   | +5.3455e+04 | 4.90e+01 | 7.39e+02 | 0.00e+00 | 1.00e+00 |  1.00e-01   |   2   |
|   4   | 64520 |   4   | +2.6740e+02 | 4.90e+01 | 2.12e-01 | 0.00e+00 | 1.00e+00 |  1.00e-01   |   4   |
|   5   | 80650 |   5   | +1.3364e+00 | 4.90e+01 | 4.06e-04 | 0.00e+00 | 1.00e+00 |  1.00e-01   |   4   |
|   6   | 96780 |   6   | +1.0000e-01 | 4.90e+01 | 4.10e-04 | 0.00e+00 | 1.00e+00 |  1.00e-01   |   4   |
|   7   | 96780 |   6   | +1.0000e-01 | 2.45e+02 | 8.20e-05 | 0.00e+00 | 1.00e+00 |  2.00e-02   |   0   |
|   8   |112910 |   7   | +2.0187e-02 | 2.45e+

In [9]:
Laplacian = sol.x

In [10]:
Laplacian

array([[ 1.58162706, -0.27193459, -0.10418694, ..., -0.01297106,
        -0.01623607, -0.01450065],
       [-0.27196929,  1.57969429, -0.26976703, ..., -0.01268548,
        -0.01556632, -0.01451765],
       [-0.10418739, -0.2697845 ,  1.58101302, ..., -0.01237394,
        -0.01488159, -0.01429582],
       ...,
       [-0.01308682, -0.01264466, -0.01231483, ...,  1.51658466,
        -0.31138079, -0.31144413],
       [-0.01623581, -0.01554696, -0.01468744, ..., -0.31122808,
         1.51798148, -0.3109655 ],
       [-0.01450065, -0.01436183, -0.0144154 , ..., -0.31160103,
        -0.31112654,  1.51869206]])